# Week 1

In [ ]:
#~source: https: //en.wikipedia.org/wiki/Gradient_descent
# code source: https://en.wikipedia.org/w/index.php?title=Gradient_descent&oldid=966271567

# 初始值
next_x = 6# We start the search at x = 6
# 步长系数（学习率）
gamma = 0.01# Step size multiplier
# 提前停止（系数变化小于此值时停止）
precision = 0.00001# Desired precision of result
# 最大迭代次数
max_iters = 10000# Maximum number of iterations

# Derivative
#function
#求导
def df(x):
  return 4 * x ** 3 - 9 * x ** 2

# 迭代
for i in range(max_iters):
    current_x = next_x
    
    #梯度下降
    next_x = current_x - gamma * df(current_x)
    print(i, next_x, df(current_x))

    # 提前停止的判定，计算系数变化了多少，小于阈值后提前停止
    step = next_x - current_x
    if abs(step) <= precision:
        break

print("Minimum at ", next_x)

# The output for the above will be something like 
# "Minimum at 2.2499646074278457"

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample
# y = mx + b
# m is slope, b is y-intercept
# 计算均方误差 MSE
def compute_error_for_line_given_points(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        # 实际的y减去 mx+b 的 y
        totalError += (y - (m * x + b)) ** 2
    return totalError / float(len(points))

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample

# 梯度下降用于线性回归
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    N = float(len(points))
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        
        # 负梯度计算 （Loss Function 对 y_pre的导数，乘以 y_pred 对 m和b的导数
        # 除以N是因为有N个样本，相当于取平均值
        b_gradient += -(2/N) * (y - ((m_current * x) + b_current))
        m_gradient += -(2/N) * x * (y - ((m_current * x) + b_current))
    new_b = b_current - (learningRate * b_gradient)
    new_m = m_current - (learningRate * m_gradient)
    return [new_b, new_m]

In [ ]:
# 执行梯度下降的函数
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, num_iterations):
    b = starting_b
    m = starting_m
    for i in range(num_iterations):
        b, m = step_gradient(b, m, array(points), learning_rate)
    return [b, m]

In [ ]:
# 用上面的代码合到一起，跑起来
def run():
    points = genfromtxt("data_linearreg.csv", delimiter=",")
    learning_rate = 0.0001
    initial_b = 0 # initial y-intercept guess
    initial_m = 0 # initial slope guess
    num_iterations = 1000
    print ("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_error_for_line_given_points(initial_b, initial_m, points)))
    print ("Running...")
    [b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, num_iterations)
    print ("After {0} iterations b = {1}, m = {2}, error = {3}".format(num_iterations, b, m, compute_error_for_line_given_points(b, m, points)) )

In [ ]:
#source: https://stackoverflow.com/questions/3949226/calculating-pearson-correlation-and-significance-in-python

import math
import numpy as np
from random import random

# 计算 R Score
def pcc(X, Y):
   ''' Compute Pearson Correlation Coefficient. '''
   # Normalise X and Y
   X -= X.mean(0)
   Y -= Y.mean(0)
   # Standardise X and Y
   X /= X.std(0)
   Y /= Y.std(0)
   # Compute mean product
   return np.mean(X*Y)
 
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    # 协方差 除以 两个标准差的乘积
    return diffprod / math.sqrt(xdiff2 * ydiff2)

#main

# Using it on a random example

X = np.array([random() for x in range(100)])
Y = np.array([random() for x in range(100)])


# 两种等效的写法
pcof = pcc(X, Y)
print(pcof, ' is pcof')
 
pcoftwo = pearson_def(X, Y)
print(pcoftwo, ' is pcof second version')